In [1]:
# Imports
# %matplotlib inline
# %config InlineBackend.figure_format = 'svg'
import msprime  # >= 0.5
import numpy as np
import math
# import scipy
import seaborn as sns
import pandas as pd
# import multiprocessing
import matplotlib.pyplot as plt
from IPython.display import SVG
# import os
# from scipy import stats
import itertools
import tskit
import demes
import demesdraw

In [19]:
def snm(N, seq_len, num_replicates, mu, ro):
    demography = msprime.Demography()
    demography.add_population(
        name="pop0",
        description="Single population",
        initial_size=N,
        growth_rate=0,
    )

    demography.sort_events()
    
    ancestry_reps = msprime.sim_ancestry(
        {"pop0": 10},
        demography=demography,
        sequence_length = seq_len,
        recombination_rate = ro,
        num_replicates=num_replicates)
    
    for ts in ancestry_reps:
        mutated_ts = msprime.sim_mutations(ts, rate=mu)
        yield mutated_ts

In [20]:
def two_epoch(N, N2, seq_len, num_replicates, mu, ro):
    demography = msprime.Demography()
    demography.add_population(
        name="pop0",
        description="Single population",
        initial_size=N2,
        growth_rate=0,
    )
    
    demography.add_population_parameters_change(time=N2, initial_size=N, growth_rate=None, population='pop0')

    demography.sort_events()
    
    ancestry_reps = msprime.sim_ancestry(
        {"pop0": 10},
        demography=demography,
        sequence_length = seq_len,
        recombination_rate = ro,
        num_replicates=num_replicates)
    
    for ts in ancestry_reps:
        mutated_ts = msprime.sim_mutations(ts, rate=mu)
        yield mutated_ts

In [8]:
outPath = r"/method_testing/WF/"
N = 10000
ro = 1e-8
mu = 2.5e-8
seq_len = 1000000
num_replicates = 100
ts_list = []
for replicate_index, ts in enumerate(snm(N, seq_len, num_replicates, mu, ro)):
    ts_list.append(ts)
    
for rep, ts in enumerate(ts_list):
    #ts.dump(outPath + "/scaffold" + str(region) + "_rep" + str(rep) + ".ts")
    out = open(outPath + "rep" + str(rep) + ".vcf", "w")
    tskit.TreeSequence.write_vcf(ts, out)

In [22]:
d = {'expansion':20000, 'contraction':5000, 'bneck':1000}

for demog in d.keys():
    outPath = r"/method_testing/WF/"+demog+"/rr_fixed_mu_fixed/WF/"
    N = 10000
    N2 = d[demog]
    ro = 1e-8
    mu = 2.5e-8
    seq_len = 1000000
    num_replicates = 100
    ts_list = []
    for replicate_index, ts in enumerate(two_epoch(N, N2, seq_len, num_replicates, mu, ro)):
        ts_list.append(ts)

    for rep, ts in enumerate(ts_list):
        #ts.dump(outPath + "/scaffold" + str(region) + "_rep" + str(rep) + ".ts")
        out = open(outPath + "rep" + str(rep) + ".vcf", "w")
        tskit.TreeSequence.write_vcf(ts, out)